In [133]:
import pandas as pd
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import numpy as np

In [143]:
#meal_info = pd.read_csv('train_dataset/meal_info.csv')
#fulfilment_center_info = pd.read_csv('train_dataset/fulfilment_center_info.csv')
#train_dataset = pd.read_csv('train_dataset/train.csv')
test_dataset = pd.read_csv('test_dataset.csv')

In [46]:
#Finding different meals option available, this may affect the final training data
#meals_category = pd.DataFrame(meal_info.groupby(['category', 'cuisine']).size())

#performing label encoding on by first converting category column into category data type and then use cat.codes. 
#don't confuse with category column name with category data type
'''
meal_info["category"] = meal_info["category"].astype('category')
meal_info["category"] = meal_info["category"].cat.codes
'''

#now implementing label encoding on the cuisine column
label_encoding = preprocessing.LabelEncoder()
meal_info['cuisine'] = label_encoding.fit_transform(meal_info['cuisine'])
meal_info.head()

,meal_id,category,cuisine
0,1885,0,3
1,1993,0,3
2,2539,0,3
3,1248,0,1
4,2631,0,1


In [93]:
#Trying to fetch important information from fulfilment center info and performing the one hot encoding using pandas
'''
fulfilment_center_info.columns
fulfilment_center_info.groupby(['center_type']).size()
performing the one hot encoding on categorical column "center_type"
fulfilment_center_info = pd.get_dummies(fulfilment_center_info, columns = ["center_type"])
fulfilment_center_info.head()
'''


#performing one hot encoding on column "center_type" using sklearn
'''
label_encoding = preprocessing.LabelEncoder()
fulfilment_center_info['center_type'] = label_encoding.fit_transform(fulfilment_center_info['center_type'])
one_hot_encoding = preprocessing.OneHotEncoder(categorical_features=[3])
one_hot_encoding_matrix = one_hot_encoding.fit_transform(fulfilment_center_info).toarray()
fulfilment_center_info = pd.DataFrame(one_hot_encoding_matrix, columns=['Center_Type_A', 'Center_Type_B', 'Center_Type_C', 'Center_Id', 'city_code', 'region_code', 'op_area' ])
fulfilment_center_info.head()
'''

#Droping city_code and region_code column, performing the renaming of 'Center_Id'
'''
fulfilment_center_info = fulfilment_center_info.drop(['city_code', 'region_code'], axis=1)
fulfilment_center_info = fulfilment_center_info.rename(columns={'Center_Id' : 'center_id'})
fulfilment_center_info.head()
'''



,Center_Type_A,Center_Type_B,Center_Type_C,center_id,op_area
0,1.0,0.0,0.0,11.0,3.7
1,0.0,1.0,0.0,13.0,6.7
2,0.0,0.0,1.0,124.0,4.0
3,1.0,0.0,0.0,66.0,4.1
4,0.0,0.0,1.0,94.0,3.6


In [113]:
'''
preprocessed_train_dataset = train_dataset
preprocessed_train_dataset = preprocessed_train_dataset.drop(['id'], axis=1)
'''

#calculating the discount from base and checkout price and droping base, checkout and week column
'''
preprocessed_train_dataset['discount'] = preprocessed_train_dataset['base_price'] - preprocessed_train_dataset['checkout_price']
preprocessed_train_dataset = preprocessed_train_dataset.drop(['checkout_price', 'base_price'], axis = 1)
preprocessed_train_dataset = preprocessed_train_dataset.drop(['week'], axis = 1)
'''

#merging dataframe 'fulfilment_center_info' and 'preprocessed_train_dataset'
'''
merge_preprocessed_train_dataset = pd.merge(fulfilment_center_info, preprocessed_train_dataset,how='outer')
merge_preprocessed_train_dataset
'''

#merging dataframe 'meal_info' and 'merge_pre_preprocessed_train_dataset'
'''
merge_preprocessed_train_dataset = pd.merge(meal_info, merge_preprocessed_train_dataset, how='outer')
merge_preprocessed_train_dataset
'''

#now droping the unnecessary column 'meal_id' and 'center_id'
merge_preprocessed_train_dataset = merge_preprocessed_train_dataset.drop(['meal_id', 'center_id'], axis = 1)
merge_preprocessed_train_dataset.head()

,category,cuisine,Center_Type_A,Center_Type_B,Center_Type_C,op_area,emailer_for_promotion,homepage_featured,num_orders,discount
0,0,3,1.0,0.0,0.0,3.7,0,0,959,0.0
1,0,3,1.0,0.0,0.0,3.7,0,0,1054,2.0
2,0,3,1.0,0.0,0.0,3.7,0,0,891,-2.0
3,0,3,1.0,0.0,0.0,3.7,0,0,1107,1.0
4,0,3,1.0,0.0,0.0,3.7,0,0,919,0.0


In [125]:
# now going to create the lasso regression model
X_Train, X_Val, Y_Train, Y_Val = train_test_split(merge_preprocessed_train_dataset[['category', 'cuisine', 'Center_Type_A', 'Center_Type_B', 'Center_Type_C', 'op_area', 'emailer_for_promotion', 'homepage_featured', 'discount']], merge_preprocessed_train_dataset['num_orders'], test_size = 0.3, random_state =42)


lasso_regression_model = linear_model.Lasso()
lasso_regression_model.fit(X_Train, Y_Train)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [142]:
#now checking the prediction 
'''
prediction_value = lasso_regression_model.predict(X_Val)
prediction_value = np.array(prediction_value)
prediction_value = np.absolute(prediction_value)
prediction_value = prediction_value.astype(int)
'''

mean_square_log_error_value = mean_squared_log_error(Y_Val, prediction_value)


1.5660121892211627

In [159]:
#calculating the prediction test samples

#removing the id column
'''
preprocessed_test_dataset = test_dataset
preprocessed_test_dataset = preprocessed_test_dataset.drop(['id'], axis=1)
'''

#calculating the discount from base and checkout price and droping base, checkout and week column
'''
preprocessed_test_dataset['discount'] = preprocessed_test_dataset['base_price'] - preprocessed_test_dataset['checkout_price']
preprocessed_test_dataset = preprocessed_test_dataset.drop(['checkout_price', 'base_price'], axis = 1)
preprocessed_test_dataset = preprocessed_test_dataset.drop(['week'], axis = 1)
'''

#merging dataframe 'fulfilment_center_info' and 'preprocessed_test_dataset'
'''
merge_preprocessed_test_dataset = pd.merge(fulfilment_center_info, preprocessed_test_dataset,how='outer')
print(merge_preprocessed_test_dataset)
'''

#merging dataframe 'meal_info' and 'merge_pre_preprocessed_test_dataset'
'''
merge_preprocessed_test_dataset = pd.merge(meal_info, merge_preprocessed_test_dataset, how='outer')
print(merge_preprocessed_test_dataset)
'''

#now droping the unnecessary column 'meal_id' and 'center_id'
'''
merge_preprocessed_test_dataset = merge_preprocessed_test_dataset.drop(['meal_id', 'center_id'], axis = 1)
merge_preprocessed_test_dataset.head()
'''

#calculating the final prediction value for each order
'''
final_prediction_value = lasso_regression_model.predict(merge_preprocessed_test_dataset)
final_prediction_value = np.array(final_prediction_value)
final_prediction_value = np.absolute(final_prediction_value)
final_prediction_value = final_prediction_value.astype(int)
'''

#creating the submission format
#test_dataset_id = test_dataset.loc[:, 'id']
#submission = pd.DataFrame(list(zip(test_dataset_id, final_prediction_value)), columns = ['id', 'num_orders'])
submission.to_csv('food_demand_forecasting_prediction_output.csv', index=False)